In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import os
import cv2
from joblib import dump, load
from commonfunctions import *

target_img_size = (128, 128)

In [2]:
def extract_hog_features(img):
    """
    Extracts Histogram of Oriented Gradients (HOG) features from the input image.
    This involves resizing the image to a fixed size, dividing it into cells and blocks, 
    computing gradient histograms, and flattening the result into a feature vector. 
    The extracted features are useful for machine learning tasks like image classification and detection.
    """
    img = cv2.resize(img, dsize=target_img_size)
    win_size = (128, 128) #The sliding window size over which HOG features are computed. In this case, it matches the image size (32x32)
    cell_size = (4, 4) #Divides the window into smaller cells (4x4 pixels per cell).
    block_size_in_cells = (2, 2) 

    # divides window to blocks then blocks to cells
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0]) #Determines how much the block moves at each step (4x4 pixels, matching cell size).
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten() #This array is used as input for machine learning models.

def load_model():
    # Load the digits models dictionary
    loaded_digits_models = load('digits_models.joblib')
    print('Loaded digits models from digits_models.joblib')
    print(loaded_digits_models)

    # Load the symbols models dictionary
    loaded_symbols_models = load('symbols_models.joblib')
    print('Loaded symbols models from symbols_models.joblib')

    # Return the loaded models
    return loaded_digits_models, loaded_symbols_models
    

digits_models, symbols_models = load_model()
    
def predict_symbol(img):

    test_features=extract_hog_features(img)
    predicted_symbol=symbols_models['SVM'].predict([test_features])
    return predicted_symbol



def predict_digit(img):
    test_features=extract_hog_features(img)
    predicted_digit=digits_models['SVM'].predict([test_features])
    predicted_digit_num = ord(predicted_digit[0].lower()) - ord('a') + 1
    
    if(predicted_digit == 'j'):
        predicted_digit_num = 0
    return predicted_digit_num

# test images
img=cv2.imread('test_dataset/2.png',0)
print(predict_digit(img))

img=cv2.imread('test_dataset/9.png',0)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(predict_digit(img))

img=cv2.imread('test_dataset/correct.png',0)
print(predict_symbol(img))

img=cv2.imread('test_dataset/hIIII.png',0)
print(predict_symbol(img))

Loaded digits models from digits_models.joblib
{'SVM': LinearSVC(random_state=42)}
Loaded symbols models from symbols_models.joblib


ValueError: X has 34596 features, but LinearSVC is expecting 576 features as input.